In [45]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

Load fmri extracted raw time series from cerebellum atlas

In [428]:
#discarded subjects filter
bad = pd.read_csv("bad.txt")

#features precision
precision = 'float32'
# origin h5 file location. It is used also to storage transformed data
h5_file = '/results/00-EXTRACCION-CEREBELO/elekin_fmri_ts_pandas.hdf5'
et_key = 'cbl/et' # schema ('0', '1'..'28','id')
ct_key = 'cbl/ct' # schema ('0', '1'..'28','id')
tr_key = 'cbl/dataset'#transformed dataset (id,time series matrix 158x28, label)

ct = pd.read_hdf(h5_file, ct_key)
ct['label'] = 0

et = pd.read_hdf(h5_file, et_key)
et['label'] = 1

# merge sick and control data in same dataframe
dataset = pd.concat([et,ct])
mask = dataset["id"].isin(bad["id"]).values
dataset = dataset[~mask].set_index('id')
features=dataset.columns[(dataset.columns != 'id') & (dataset.columns != 'label')] # estract feature names

In [430]:
dataset.loc[bad.values,features].groupby('id').apply(lambda x: x.values.shape)

KeyError: "None of [Index([('C024',), ('T031',), ('T060',)], dtype='object', name='id')] are in the [index]"

In [431]:
#Collect time series grouped by subject from cerebellum
features_df = pd.DataFrame(dataset.loc[:,features].groupby('id').apply(lambda x: x.values.astype(precision)), columns=['features'])

In [432]:
# Join with labels
transformed_df = features_df.join(dataset['label'].groupby('id').apply(lambda x: x.unique()[0]))

Now we transform the dataset and there is just the subject_id as dataframe row ids. Feature matrix of 158x28 where 158
corresponds with length od time series and 28 is the number of areas belonging to previously same extracted ROI (Cerebellum)

In [433]:
transformed_df.loc["T082"]["features"].shape

(158, 28)

Save to h5 file

In [434]:
transformed_df.to_hdf(h5_file, tr_key)